In [1]:
'''
STU according to GFitter 1407.3792 input for STU
'''
import numpy as np
import pandas as pd
import scipy as sp
import scipy.integrate as integrate

#GFitter 1407.3792 input for STU

sfit3=0.05
tfit3=0.09
ufit3=0.01

sigmas3=0.11
sigmat3=0.13
sigmau3=0.11

rhost3=0.90
rhosu3=-0.59
rhotu3=-0.83


st=sigmat3
ss=sigmas3
su=sigmau3

rst=rhost3
rsu=rhosu3
rtu=rhotu3
#
sfit=sfit3
tfit=tfit3
ufit=ufit3


# chi^2 in 2-D as a func. of Sfit,Tfit,DeltaS,DeltaT,sigmaS,sigmaT,rhoST.
def fchi2st0(s,t,sfit,tfit,ss,st,r):
    #print " example for ST, U=0. \n evaluates chi^2."
    y=((s-sfit)**2/ss**2 + (t-tfit)**2/st**2 - 2*r*(s-sfit)*(t-tfit)/(ss*st))/(1- r**2)
    return y
    pass

# CL as func. of fchi2st0().
def fcl(s,t,sfit,tfit,ss,st,r):
    y =1-np.exp(-fchi2st0(s,t,sfit,tfit,ss,st,r)/2)
    return y

#fchi2st0(0.02,0.02,sfit,tfit,st,ss,rst)


# chi^2 in 3-D as a func. of Sfit,Tfit,Ufit,DeltaS,DeltaT,DeltaU,sigmaS,sigmaT,sigmaU,rhoST,rhoSU,rhoTU.

#detM, needed to invert M.
#detMred=detM/(ss*st*su)**2

def detMred(ss,st,su,rst,rsu,rtu):
    y = 1 -rst**2-rsu**2-rtu**2 + 2*rst*rsu*rtu
    return y

def fchi3stu(s,t,u,sfit,tfit,ufit,ss,st,su,rst,rsu,rtu):
    #print " example for ST, U=0. \n evaluates chi^2."
    y=((s-sfit)**2*(1-rtu**2)/ss**2 + (t-tfit)**2*(1-rsu**2)/st**2  +\
       (u-ufit)**2*(1-rst**2)/su**2 - 2*(s-sfit)*(t-tfit)*(rst-rsu*rtu)/(ss*st)-\
       2*(s-sfit)*(u-ufit)*(rsu-rst*rtu)/(ss*su)- 2*(t-tfit)*(u-ufit)*(rtu-rst*rsu)/(st*su))\
        /detMred(ss,st,su,rst,rsu,rtu)
    return y


# CL as func. of fchi3stu().
def fcl3(s,t,u,sfit,tfit,ufit,ss,st,su,rst,rsu,rtu):
    y =1-np.exp(-fchi3stu(s,t,u,sfit,tfit,ufit,ss,st,su,rst,rsu,rtu)/2)
    return y




# S, T and U oblique parameters '
def f(x,p,m1,m2):
    ''' For STU parameters
    defined according to arxiv 1408.0961v1 in the limit in which
    cos\theta= 1.0 [see the definition in the given arxiv reference]'''
    y = ((2.0*x - 1.0)*(m1**2 - m2**2) + (2*x-1)**2*p**2)*np.log(x*m1**2 + (1-x)*m2**2 - x*(1-x)*p**2)
    return y



def S(dp,SM,cl=[0.68,0.95,0.977]):
    '''
    Input parameters 
    1. dp. --> Dictionary where the input parameters are defined.
    This dictionary will be defined later on
    
    2. cps. --> Dictionary where internal parameters are defined. 
      S,T and U
      fcl3. Confidence level for STU
    '''
    cps=pd.Series()
    dp=pd.Series(dp)
    SM=pd.Series(SM)

    F1=(SM.MZ,dp.MHC,dp.MHC)
    F2=(SM.MZ,dp.MHX,dp.MH3)
    F3=(SM.MW,dp.MHC,dp.MHX)
    F4=(SM.MW,dp.MHC,dp.MH3)
    F5=(0,dp.MHX,dp.MH3)
    F6=(0,dp.MHC,dp.MHX)
    F7=(0,dp.MHC,dp.MH3)
    F8=(0,dp.MH3,dp.MHX)
    
    
    cps['S']=1.0/(4*np.pi*(SM.MZ)**2)*((1.0-2.0)*integrate.quad(f,0,1,args=F1)[0]+(integrate.quad(f,0,1,args=F2)[0]-integrate.quad(f,0,1,args=F5)[0] ))
    cps['T']=1.0/(16*np.pi**2*(SM.alpha_em)*(SM.vev)**2)*(( integrate.quad(f,0,1,args=F6)[0]+integrate.quad(f,0,1,args=F7)[0] -integrate.quad(f,0,1,args=F8)[0]))
    cps['U']=-1.0/(4*np.pi*(SM.MZ)**2)*(integrate.quad(f,0,1,args=F1)[0]+(integrate.quad(f,0,1,args=F2)[0]-integrate.quad(f,0,1,args=F5)[0])-\
                ((SM.MZ)**2/(SM.MW)**2)*((integrate.quad(f,0,1,args=F3)[0]-integrate.quad(f,0,1,args=F6)[0]+integrate.quad(f,0,1,args=F4)[0]-\
                                        integrate.quad(f,0,1,args=F7)[0])))
        
    cps['fcl3']=fcl3(cps['S'],cps['T'],cps['U'],sfit,tfit,ufit,ss,st,su,rst,rsu,rtu)
    cps['cl']=cl[2]
    return cps


In [2]:
SM={'alpha_em': 1./128.,'vev': 246.0, 'MW': 80.385, 'MZ': 91.1876}
dp={'MHX': 902., 'MH3':705., 'MHC':700., 'Mh':125.}  
cps=S(dp,SM)

print cps

S       0.007566
T       0.017920
U       0.000056
fcl3    0.735910
cl      0.977000
dtype: float64
